# Banknote Authentication Model



## Introduction:
    This project is created for checking multiple classifiers on the UCI Banknote Authentication Data Set( https://archive.ics.uci.edu/ml/datasets/banknote+authentication). 

In [16]:
from IPython.display import IFrame
IFrame('https://archive.ics.uci.edu/ml/datasets/banknote+authentication', width=800, height=450)

## Objective:
    Main objective of this project is to evaluate, which classifier provides the best optimal solution for classifying whether a note is authentic or not.

## Importing Necessary Packages
    We are going to use python for this project. so first step is to getting all the necessary pckages

In [17]:
import numpy as np
np.__version__

'1.16.5'

In [18]:
import pandas as pd # for loading the CSV data
pd.__version__

'0.25.1'

In [19]:
from sklearn import preprocessing # for spliting data into Training set and Test set

## Loding The DataSet
    The data set is in CSV(Comma seperated Value) format. We are using pandas for importing this dataset

In [20]:
features_name= ["Variance", "Skewness", "Curtosis", "Entropy", "Class"]  # These are the features list for our dataset
banknote_data_set= pd.read_csv('data_banknote_authentication.csv', names= features_name)
banknote_data_set.head(5) # geting a snapshot of our data

,Variance,Skewness,Curtosis,Entropy,Class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


### Non-Numeric Values
    Our dataset dosen't contain any Non-Numeric value so we don't have to worry about that.


### Null Value
    Our dataset dose't have any null value so we don't have to wory about that.

### Standardization & Normalization
    Our dataset contains continious values so the values in all the columns are not in same range so we are performing Normalization and the scale across all the data are also not same so one value may get uncondition prority than others thats why we are performing Standardization.

In [21]:
print("{0:2}\t{1:20}\t{2:10}".format("No", "Mean", "Std"))
for column_name in features_name[0:3]:  # for all the columns except the last one
  column_mean= banknote_data_set[column_name].mean()
  print("{0:2d}\t{1}".format( features_name.index(column_name) + 1, column_mean), end="\t")

  column_std= banknote_data_set[column_name].std()
  print("{}".format(column_std))

  banknote_data_set[column_name]= (banknote_data_set[column_name]-column_mean)/column_std

banknote_data_set.head(5)

No	Mean                	Std       
 1	0.4337352570699707	2.842762586278562
 2	1.9223531206393603	5.869046743695522
 3	1.3976271172667651	4.310030090106595


,Variance,Skewness,Curtosis,Entropy,Class
0,1.121397,1.149036,-0.975614,-0.44699,0
1,1.446538,1.064065,-0.894710,-1.46210,0
2,1.207369,-0.777069,0.122174,0.10645,0
3,1.063355,1.295005,-1.254940,-3.59440,0
4,-0.036758,-1.086642,0.736462,-0.98880,0


### Spliting the features and class colum
    The imported data have all it's values in the same data frame table witch is hard to work with, so we are spliting it into two parts one is for features an another is for output classes.

### Converting The Data Frame to ndArray
    Working with pandas data frame is problematic and it may produce inconcistency with sklearn packages so we are converting the Data Frame to ndArray.

In [22]:
features= banknote_data_set[features_name[0:3]].to_numpy()
classes= banknote_data_set[features_name[-1]].to_numpy()

print("Features ndArray: ", features, end="\n\n")
print("Classes ndArray: ", classes)

Features ndArray:  [[ 1.12139676  1.14903615 -0.97561433]
 [ 1.44653822  1.06406494 -0.89471002]
 [ 1.20736947 -0.7770688   0.12217383]
 ...
 [-1.47182015 -2.62069017  3.75764729]
 [-1.40617978 -1.75583081  2.55111279]
 [-1.04674069 -0.43966137  0.29850671]]

Classes ndArray:  [0 0 0 ... 1 1 1]
